# SMART-G validation IPRT phase B - Cubic cloud (C2)
- https://www.meteo.physik.uni-muenchen.de/~iprt/doku.php?id=start
- Be careful !! This is not a final notebook.
  The modules, functions, classes related to the 3D atm mode may drastically change in the new release!


In [ ]:
%matplotlib inline
# next 2 lines allow to automatically reload modules that have been changed externally
%load_ext autoreload
%autoreload 2

import os, sys

try:
    import subprocess
    check = subprocess.check_call(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    ROOTPATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8') # Root Git Path
except subprocess.CalledProcessError:
    ROOTPATH = os.getcwd()
sys.path.insert(0, ROOTPATH)

from smartg.config import DIR_AUXDATA
from smartg.libATM3D import satellite_view, Grid3D, Cloud3D, Atm3D, create_sensors, read_cld_nth_cte
from smartg.smartg import Smartg, LambSurface, Albedo_cst
from smartg.atmosphere import AtmAFGL, diff1
import pandas as pd
import numpy as np
from smartg.iprt.iprt import compute_deltam, groupIQUV

S3DB = Smartg(opt3D=True, alt_pp=True, alis=False, back=True, double=True, bias=True)
S3DF = Smartg(opt3D=True, alt_pp=True, alis=False, back=False, double=True, bias=True)
NBPHOTONS = 49e9
NBLOOP    = 1e8

# In case we want to find the optimal number of block and grid (CUDA) for each sim
# The values bellow can (have may be to) be modified (depending on the GPU)
FIND_OPTIMAL_XB_XG = False
XB = [32, 64, 128]
XG = [512, 1024]
CHECK_NBLOOP = 1e8
CHECK_NBPHOTONS = 1e8

# Else take the following number of block and grid (values accepted by most of GPUs after 10xx series)
XBLOCK = 128
XGRID = 1024

## Without atmosphere

### Commun to all cases

#### Atmosphere profil

In [ ]:
scale = 1 # can be useful for grid with very small cells

# ========= phase matrix
NTH = 18001 # 1801 is not enough for case 6, then we take 18001
file_cld_phase = DIR_AUXDATA + "/IPRT/phaseB/opt_prop/watercloud_800.mie.cdf"
cld_phase = read_cld_nth_cte(filename=file_cld_phase, nb_theta=NTH)

# ========= grid
# ********* method reduced grid = faster *********
xgrid = np.array([0., 3., 4., 7.])*scale
ygrid = np.array([0., 3., 4., 7.])*scale
zgrid = np.array([0., 2., 3., 5.])*scale
grid3 = Grid3D(xgrid, ygrid, zgrid, periodic=True)
# First column x, second y and third z
# We follow the IPRT convention for indices (start at 1 instead of 0)
# the cubic cloud is between 3 and 4 km in x and y, and between 2 and 3 km in z.
cloud_indices1 = np.zeros((1,3), dtype=np.int32)
cloud_indices1[0,:] = np.array([2, 2, 2]) #, x, y and z indices according to x, y and z grids
cld_ext_coeff1 = np.zeros(1, dtype=np.float64)
cld_ext_coeff1[0] = 10.
reff = np.zeros_like(cld_ext_coeff1, dtype=np.float64)
reff[0] = 10.
cloud3 = Cloud3D('wc', w_ref=800., ext_ref=cld_ext_coeff1, xyz_grids=[grid3.xGRID, grid3.yGRID, grid3.zGRID],
                 cell_indices=cloud_indices1, reff=reff, phase=cld_phase)
# *****************************************************************************************************************

# ********* method complete grid = slower but same grid as in IPRT paper *********
# cubic_cloud_f= DIR_AUXDATA +'/IPRT/phaseB/grids/C2_cloud_70x70x5.dat'
# cloud3 = Cloud3D('wc', w_ref=800., ext_reff_filename=cubic_cloud_f, phase=cld_phase)
# xgrid, ygrid, zgrid = cloud3.get_xyz_grid(loc_xgrid=0, loc_ygrid=0)
# grid3 = Grid3D(xgrid*scale, ygrid*scale, zgrid*scale, periodic=True)
# *****************************************************************************************************************

atm3 = Atm3D('afglt', grid3, wls=np.array([800.]), wl_ref= 800., cloud_3d=cloud3, tauR=0., NO2=False, O3=0., H2O=0.)

grid = atm3.get_grid()
prof_ray = atm3.get_glob_molecular_sca() # Rayleigh
prof_abs = atm3.get_glob_molecular_abs()

ext_aer = atm3.get_glob_aer_ext()*(1/scale)
ssa_aer = np.ones_like(ext_aer) # ssa forced to 1, must have the same form as ext_cld3D
prof_aer = (ext_aer, ssa_aer)

prof_phases = atm3.get_glob_aer_phase(wl_phase=[800.], NBTHETA=NTH)

cells = atm3.get_cells_info()

### profiles computations
atm3D = AtmAFGL('ATM3D', US=False,
                    grid        = grid,
                    prof_ray    = prof_ray,
                    prof_abs    = prof_abs,
                    prof_aer    = prof_aer,
                    prof_phases = prof_phases,
                    cells       = cells
                   )
pro3D3_C2_noatm  = atm3D.calc(atm3.wls, NBTHETA=NTH)

surf_C2 = LambSurface(ALB=Albedo_cst(0.2))

#### Function to print results

In [ ]:
def print_C2_res_noatm(m, norm, TCASE, grid3_sensors, U_sign=1, v_sign=-1, mI=None, mQ=None, mU=None, mV=None):
    
    if mI is None: I_SMARTG = m["I_up (TOA)"][:,0,0].reshape(70,70)*norm
    else:  I_SMARTG = mI.reshape(70,70)*norm
    # I_SMARTG_stdev = m["I_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm
    if mQ is None: Q_SMARTG = m["Q_up (TOA)"][:,0,0].reshape(70,70)*norm
    else:  Q_SMARTG = mQ.reshape(70,70)*norm
    # Q_SMARTG_stdev = m["Q_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm
    if mU is None: U_SMARTG = m["U_up (TOA)"][:,0,0].reshape(70,70)*norm*U_sign
    else:  U_SMARTG = mU.reshape(70,70)*norm*U_sign
    # U_SMARTG_stdev = m["U_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm
    if mV is None: V_SMARTG = m["V_up (TOA)"][:,0,0].reshape(70,70)*norm*v_sign
    else:  V_SMARTG = mV.reshape(70,70)*norm*v_sign
    # V_SMARTG_stdev = m["V_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm



    file_res = DIR_AUXDATA + "/IPRT/phaseB/mystic_res/iprt_case_C2_mystic.dat"
    read_res = pd.read_csv(file_res, skiprows = (4900*(TCASE-1)) + 3, nrows = 4900, header=None, sep='\s+', dtype=float).values
    I_MYSTIC = read_res[:,7].reshape(70,70).T
    # I_MYSTIC_stdev = read_res[:,11].reshape(70,70).T
    Q_MYSTIC = read_res[:,8].reshape(70,70).T
    # Q_MYSTIC_stdev = read_res[:,12].reshape(70,70).T
    U_MYSTIC = read_res[:,9].reshape(70,70).T
    # U_MYSTIC_stdev = read_res[:,13].reshape(70,70).T
    V_MYSTIC = read_res[:,10].reshape(70,70).T
    # V_MYSTIC_stdev = read_res[:,14].reshape(70,70).T


    stk = ['I', 'Q', 'U', 'V']
    wl = m.axes['wavelength']
    xgrid = grid3_sensors.xgrid
    ygrid = grid3_sensors.ygrid
    interp_name = 'none'
    fig_size = (10.5,7)
    font_size=int(16)
    cb_shrink = 1
    color_bar = ['jet', 'coolwarm', 'coolwarm', 'coolwarm']
    vmin = [np.min(np.abs(I_SMARTG)), -np.max(np.abs(Q_SMARTG)), -np.max(np.abs(U_SMARTG)), -np.max(np.abs(V_SMARTG))]
    vmax = [np.max(I_SMARTG), np.max(np.abs(Q_SMARTG)), np.max(np.abs(U_SMARTG)), np.max(np.abs(V_SMARTG))]
    color_bar_std = ['coolwarm', 'coolwarm', 'coolwarm', 'coolwarm']
    vmin_std = [-np.max(I_SMARTG)*0.05, -np.max(np.abs(Q_SMARTG))*0.05, -np.max(np.abs(U_SMARTG))*0.05, -np.max(np.abs(V_SMARTG))*0.015]
    vmax_std = [np.max(I_SMARTG)*0.05, np.max(np.abs(Q_SMARTG))*0.05, np.max(np.abs(U_SMARTG))*0.05, np.max(np.abs(V_SMARTG))*0.015]


    factor = None
    mat_force = [I_SMARTG, Q_SMARTG, U_SMARTG, V_SMARTG]
    satellite_view(m, xgrid, ygrid, wl, interp_name,
                color_bar, fig_size=fig_size, font_size=font_size,
                vmin = vmin, vmax = vmax, scale=False, save_file=None, stk=stk, factor=factor,
                mat_force=mat_force, cb_shrink=cb_shrink, cb_sform=True, fig_title=f"C2 - case {TCASE} - SMART-G - without atm")

    factor = None
    mat_force = [I_SMARTG-I_MYSTIC, Q_SMARTG-Q_MYSTIC, U_SMARTG-U_MYSTIC, V_SMARTG-V_MYSTIC]
    satellite_view(m, xgrid, ygrid, wl, interp_name,
                color_bar_std, fig_size=fig_size, font_size=font_size,
                vmin = vmin_std, vmax = vmax_std, scale=False, save_file=None, stk=stk, factor=factor,
                mat_force=mat_force, cb_shrink=cb_shrink, cb_sform=True, fig_title=f"C2 - case {TCASE} - dif(SMART-G - MYSTIC) - without atm")
    

    
    # print deltam
    IQUV_SMARTG = groupIQUV(lI=[I_SMARTG], lQ=[Q_SMARTG], lU=[U_SMARTG], lV=[V_SMARTG])
    IQUV_MYSTIC = groupIQUV(lI=[I_MYSTIC], lQ=[Q_MYSTIC], lU=[U_MYSTIC], lV=[V_MYSTIC])

    print("SMART-G (delta_m):")
    delta_m = compute_deltam(obs=IQUV_MYSTIC, mod=IQUV_SMARTG, print_res=True)

### Backward simulations

#### Case 1

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 0.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180.
# count_level = 0 -> only COUNT TOA (default value = -2, i.e., count everything)
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_1_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_1_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_1_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_1_noatm
norm = norm_C2_1_noatm
TCASE = int(1)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 2

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 60.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_2_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_2_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_2_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_2_noatm
norm = norm_C2_2_noatm
TCASE = int(2)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 3

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 120.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_3_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_3_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_3_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_3_noatm
norm = norm_C2_3_noatm
TCASE = int(3)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 4

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 180.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_4_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_4_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_4_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_4_noatm
norm = norm_C2_4_noatm
TCASE = int(4)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 5

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 180.
PHI = 0.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_5_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_5_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_5_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_5_noatm
norm = norm_C2_5_noatm
TCASE = int(5)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 6

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 0.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_6_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_6_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_6_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_6_noatm
norm = norm_C2_6_noatm
TCASE = int(6)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 7

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 60.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_7_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_7_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_7_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_7_noatm
norm = norm_C2_7_noatm
TCASE = int(7)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 8

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 120.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_8_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_8_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_8_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_8_noatm
norm = norm_C2_8_noatm
TCASE = int(8)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

#### Case 9

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 180.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_9_noatm = S3DB.run(wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_noatm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True)
norm_C2_9_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_9_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_9_noatm
norm = norm_C2_9_noatm
TCASE = int(9)

print_C2_res_noatm(m, norm, TCASE, grid3_sensors)

### Forward simulations

#### Case 1 - 4

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6

print("POSZ = ", POSZ)

# ======= Case 1 - 4
    # Sun position
THETA_0    = 20.
PHI_0    = 180.

THETA_0_bis = 180.- THETA_0
PHI_0_bis = 180.- PHI_0

# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA_0_bis, PHDEG=PHI_0_bis, FOV=0., LOC='ATMOS',
                                        CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

THETA = np.array([40., 40., 40., 40.])
PHI = np.array([0., 60., 120., 180.])

le     = {'th_deg':np.array(THETA),
        'phi_deg':np.array(PHI+180.),
        'count_level':np.array([1, 1, 1, 1]),
        'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DF.run(THVDEG=THETA_0, wl=pro3D3_C2_noatm.axes['wavelength'],
                                    NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                    sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                                    XBLOCK = xblock, XGRID = xgrid, progress=False,
                                    OUTPUT_LAYERS=3)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_1to4_F_noatm = S3DF.run(THVDEG=THETA_0, wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP,
                             atm=pro3D3_C2_noatm, sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                             XBLOCK = xBLOCK, XGRID = xGRID, OUTPUT_LAYERS=3)
norm_C2_1to4_F_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_1to4_F_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_1to4_F_noatm
norm = norm_C2_1to4_F_noatm
for i in range (0, 4):
    TCASE = int(i+1)
    indZA = round(TCASE - 1)
    print("TESTCASE:", TCASE)
    print_C2_res_noatm(m, norm, TCASE, grid3_sensors, U_sign=-1, v_sign=1,
                    mI=m['I_down (0+)'][:,indZA], mQ=m['Q_down (0+)'][:,indZA],
                    mU=m['U_down (0+)'][:,indZA], mV=m['V_down (0+)'][:,indZA])

#### Case 5 -9

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6

print("POSZ = ", POSZ)

# Sun position
THETA_0    = 40.
PHI_0    = 180.

THETA_0_bis = 180.- THETA_0
PHI_0_bis = 180.- PHI_0

# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA_0_bis, PHDEG=PHI_0_bis, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

THETA = np.array([180., 140., 140., 140., 140.])
PHI = np.array([0., 0., 60., 120., 180.])

le     = {'th_deg':np.array(180.-THETA),
        'phi_deg':np.array(PHI+180.),
        'count_level':np.array([0, 0, 0, 0, 0]),
        'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DF.run(THVDEG=THETA_0, wl=pro3D3_C2_noatm.axes['wavelength'],
                                    NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_noatm,
                                    sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                                    XBLOCK = xblock, XGRID = xgrid, progress=False,
                                    OUTPUT_LAYERS=1)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_5to9_F_noatm = S3DF.run(THVDEG=THETA_0, wl=pro3D3_C2_noatm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP,
                atm=pro3D3_C2_noatm, sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                XBLOCK = xBLOCK, XGRID = xGRID, OUTPUT_LAYERS=1)
norm_C2_5to9_F_noatm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_5to9_F_noatm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_5to9_F_noatm
norm = norm_C2_5to9_F_noatm
for i in range (0, 5):
    TCASE = int(i+5)
    indZA = round(TCASE - 5)
    print("TESTCASE:", TCASE)
    print_C2_res_noatm(m, norm, TCASE, grid3_sensors, U_sign=-1, v_sign=1,
                    mI=m['I_up (TOA)'][:,indZA], mQ=m['Q_up (TOA)'][:,indZA],
                    mU=m['U_up (TOA)'][:,indZA], mV=m['V_up (TOA)'][:,indZA])

## With atmosphere

### commun to all cases

#### Atmosphere profil

In [ ]:
scale = 1 # can be useful for grid with very small cells

# ========= phase matrix
NTH = 18001 # 1801 is not enough for case 6, then we take 18001
file_cld_phase = DIR_AUXDATA + "/IPRT/phaseB/opt_prop/watercloud_800.mie.cdf"
cld_phase = read_cld_nth_cte(filename=file_cld_phase, nb_theta=NTH)

# ========= grid
# ********* method reduced grid = faster *********
xgrid = np.array([0., 3., 4., 7.])*scale
ygrid = np.array([0., 3., 4., 7.])*scale
zgrid = np.array([0., 2., 3., 5.])*scale
grid3 = Grid3D(xgrid, ygrid, zgrid, periodic=True)
# First column x, second y and third z
# We follow the IPRT convention for indices (start at 1 instead of 0)
# the cubic cloud is between 3 and 4 km in x and y, and between 2 and 3 km in z.
cloud_indices1 = np.zeros((1,3), dtype=np.int32)
cloud_indices1[0,:] = np.array([2, 2, 2]) #, x, y and z indices according to x, y and z grids
cld_ext_coeff1 = np.zeros(1, dtype=np.float64)
cld_ext_coeff1[0] = 10.
reff = np.zeros_like(cld_ext_coeff1, dtype=np.float64)
reff[0] = 10.
cloud3 = Cloud3D('wc', w_ref=800., ext_ref=cld_ext_coeff1, xyz_grids=[grid3.xGRID, grid3.yGRID, grid3.zGRID],
                 cell_indices=cloud_indices1, reff=reff, phase=cld_phase)
# *****************************************************************************************************************

# ********* method complete grid = slower but same grid as in IPRT paper *********
# cubic_cloud_f= DIR_AUXDATA +'/IPRT/phaseB/grids/C2_cloud_70x70x5.dat'
# cloud3 = Cloud3D('wc', w_ref=800., ext_reff_filename=cubic_cloud_f, phase=cld_phase)
# xgrid, ygrid, zgrid = cloud3.get_xyz_grid(loc_xgrid=0, loc_ygrid=0)
# grid3 = Grid3D(xgrid*scale, ygrid*scale, zgrid*scale, periodic=True)
# *****************************************************************************************************************

#sca_ray = np.array([[0., 0.1, 0.1, 0.1]])
tauR = 0.5 # total optical rayleigh depth

dz = diff1(grid3.zGRID)
tauRcs = np.cumsum((dz/grid3.zGRID[-1])*tauR).reshape(1,len(dz)) # homogeneous distri
ot = diff1(tauRcs, axis=1)
k  = abs(ot/dz)
k[np.isnan(k)] = 0
sl = slice(None,None,1)

sca_ray = k[:,sl]  # rayleigh sca coefficient
abs_gas = np.zeros_like(sca_ray)

atm3 = Atm3D('afglt', grid3, wls=np.array([800.]), wl_ref= 800., cloud_3d=cloud3,
             mol_sca_1d=sca_ray, mol_abs_1d=abs_gas)



grid = atm3.get_grid()
prof_ray = atm3.get_glob_molecular_sca() # Rayleigh
prof_abs = atm3.get_glob_molecular_abs()

ext_aer = atm3.get_glob_aer_ext()*(1/scale)
ssa_aer = np.ones_like(ext_aer) # ssa forced to 1, must have the same form as ext_cld3D
prof_aer = (ext_aer, ssa_aer)

prof_phases = atm3.get_glob_aer_phase(wl_phase=[800.], NBTHETA=NTH)

cells = atm3.get_cells_info()

### profiles computations
atm3D = AtmAFGL('ATM3D', US=False,
                    grid        = grid,
                    prof_ray    = prof_ray,
                    prof_abs    = prof_abs,
                    prof_aer    = prof_aer,
                    prof_phases = prof_phases,
                    cells       = cells
                   )
pro3D3_C2_atm  = atm3D.calc(atm3.wls, NBTHETA=NTH)

surf_C2 = LambSurface(ALB=Albedo_cst(0.2))

#### Function to print results

In [ ]:
def print_C2_res_atm(m, norm, TCASE, grid3_sensors, U_sign=1, v_sign=-1, mI=None, mQ=None, mU=None, mV=None):

    if mI is None: I_SMARTG = m["I_up (TOA)"][:,0,0].reshape(70,70)*norm
    else:  I_SMARTG = mI.reshape(70,70)*norm
    # I_SMARTG_stdev = m["I_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm
    if mQ is None: Q_SMARTG = m["Q_up (TOA)"][:,0,0].reshape(70,70)*norm
    else:  Q_SMARTG = mQ.reshape(70,70)*norm
    # Q_SMARTG_stdev = m["Q_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm
    if mU is None: U_SMARTG = m["U_up (TOA)"][:,0,0].reshape(70,70)*norm*U_sign
    else:  U_SMARTG = mU.reshape(70,70)*norm*U_sign
    # U_SMARTG_stdev = m["U_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm
    if mV is None: V_SMARTG = m["V_up (TOA)"][:,0,0].reshape(70,70)*norm*v_sign
    else:  V_SMARTG = mV.reshape(70,70)*norm*v_sign
    # V_SMARTG_stdev = m["V_stdev_up (TOA)"][:,0,0].reshape(70,70)*norm

    TCASE_bis = TCASE + int(9)
    file_res = DIR_AUXDATA + "/IPRT/phaseB/mystic_res/iprt_case_C2_mystic.dat"
    read_res = pd.read_csv(file_res, skiprows = (4900*(TCASE_bis-1)) + 3, nrows = 4900, header=None, sep='\s+', dtype=float).values
    I_MYSTIC = read_res[:,7].reshape(70,70).T
    # I_MYSTIC_stdev = read_res[:,11].reshape(70,70).T
    Q_MYSTIC = read_res[:,8].reshape(70,70).T
    # Q_MYSTIC_stdev = read_res[:,12].reshape(70,70).T
    U_MYSTIC = read_res[:,9].reshape(70,70).T
    # U_MYSTIC_stdev = read_res[:,13].reshape(70,70).T
    V_MYSTIC = read_res[:,10].reshape(70,70).T
    # V_MYSTIC_stdev = read_res[:,14].reshape(70,70).T


    stk = ['I', 'Q', 'U', 'V']
    wl = m.axes['wavelength']
    xgrid = grid3_sensors.xgrid
    ygrid = grid3_sensors.ygrid
    interp_name = 'none'
    fig_size = (10.5,7)
    font_size=int(16)
    cb_shrink = 1
    color_bar = ['jet', 'coolwarm', 'coolwarm', 'coolwarm']
    vmin = [0., -np.max(np.abs(Q_SMARTG)), -np.max(np.abs(U_SMARTG)), -np.max(np.abs(V_SMARTG))]
    vmax = [np.max(I_SMARTG), np.max(np.abs(Q_SMARTG)), np.max(np.abs(U_SMARTG)), np.max(np.abs(V_SMARTG))]
    color_bar_std = ['coolwarm', 'coolwarm', 'coolwarm', 'coolwarm']
    vmin_std = [-np.max(I_SMARTG)*0.05, -np.max(np.abs(Q_SMARTG))*0.05, -np.max(np.abs(U_SMARTG))*0.05, -np.max(np.abs(V_SMARTG))*0.05]
    vmax_std = [np.max(I_SMARTG)*0.05, np.max(np.abs(Q_SMARTG))*0.05, np.max(np.abs(U_SMARTG))*0.05, np.max(np.abs(V_SMARTG))*0.05]


    factor = None
    mat_force = [I_SMARTG, Q_SMARTG, U_SMARTG, V_SMARTG]
    satellite_view(m, xgrid, ygrid, wl, interp_name,
                color_bar, fig_size=fig_size, font_size=font_size,
                vmin = vmin, vmax = vmax, scale=False, save_file=None, stk=stk, factor=factor,
                mat_force=mat_force, cb_shrink=cb_shrink, cb_sform=True, fig_title=f"C2 - case {TCASE} - SMART-G - with atm")

    factor = None
    mat_force = [I_SMARTG-I_MYSTIC, Q_SMARTG-Q_MYSTIC, U_SMARTG-U_MYSTIC, V_SMARTG-V_MYSTIC]
    satellite_view(m, xgrid, ygrid, wl, interp_name,
                color_bar_std, fig_size=fig_size, font_size=font_size,
                vmin = vmin_std, vmax = vmax_std, scale=False, save_file=None, stk=stk, factor=factor,
                mat_force=mat_force, cb_shrink=cb_shrink, cb_sform=True, fig_title=f"C2 - case {TCASE} - dif(SMART-G - MYSTIC) - with atm")
    

    # print deltam
    IQUV_SMARTG = groupIQUV(lI=[I_SMARTG], lQ=[Q_SMARTG], lU=[U_SMARTG], lV=[V_SMARTG])
    IQUV_MYSTIC = groupIQUV(lI=[I_MYSTIC], lQ=[Q_MYSTIC], lU=[U_MYSTIC], lV=[V_MYSTIC])

    print("SMART-G (delta_m):")
    delta_m = compute_deltam(obs=IQUV_MYSTIC, mod=IQUV_SMARTG, print_res=True)

    

### Backward simulations

#### Case 1

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 0.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_1_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_1_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_1_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_1_atm
norm = norm_C2_1_atm
TCASE = int(1)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 2

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 60.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_2_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_2_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_2_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_2_atm
norm = norm_C2_2_atm
TCASE = int(2)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 3

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 120.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_3_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_3_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_3_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_3_atm
norm = norm_C2_3_atm
TCASE = int(3)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 4

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[0]

print("POSZ = ", POSZ)
THETA = 40.
PHI = 180.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 20.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_4_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_4_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_4_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_4_atm
norm = norm_C2_4_atm
TCASE = int(4)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 5

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 180.
PHI = 0.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_5_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=1e9, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_5_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_5_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_5_atm
norm = norm_C2_5_atm
TCASE = int(5)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 6

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 0.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_6_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=1e9, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_6_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_6_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_6_atm
norm = norm_C2_6_atm
TCASE = int(6)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 7

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 60.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_7_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=1e9, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_7_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_7_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_7_atm
norm = norm_C2_7_atm
TCASE = int(7)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 8

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 120.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_8_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=1e9, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_8_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_8_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_8_atm
norm = norm_C2_8_atm
TCASE = int(8)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

#### Case 9

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6*scale

print("POSZ = ", POSZ)
THETA = 140.
PHI = 180.
# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA, PHDEG=PHI, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

# Sun position
THETA_0    = 40.
PHI_0    = 180. 
le     = {'th_deg':np.array([THETA_0]), 'phi_deg':np.array([PHI_0]), 'count_level':np.array([0])}#, 'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xblock, XGRID = xgrid, progress=False, stdev=True, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_9_atm = S3DB.run(wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=1e9, NBLOOP=NBLOOP, atm=pro3D3_C2_atm,
                          sensor=sensors, le=le, surf=surf_C2, NF=NTH, XBLOCK = xBLOCK, XGRID = xGRID, stdev=True, DEPO=0)
norm_C2_9_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_9_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_9_atm
norm = norm_C2_9_atm
TCASE = int(9)

print_C2_res_atm(m, norm, TCASE, grid3_sensors)

### Forward simulations

#### Case 1 - 4

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6

print("POSZ = ", POSZ)

# ======= Case 1 - 4
    # Sun position
THETA_0    = 20.
PHI_0    = 180.

THETA_0_bis = 180.- THETA_0
PHI_0_bis = 180.- PHI_0

# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA_0_bis, PHDEG=PHI_0_bis, FOV=0., LOC='ATMOS',
                                        CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

THETA = np.array([40., 40., 40., 40.])
PHI = np.array([0., 60., 120., 180.])

le     = {'th_deg':np.array(THETA),
        'phi_deg':np.array(PHI+180.),
        'count_level':np.array([1, 1, 1, 1]),
        'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DF.run(THVDEG=THETA_0, PHVDEG=PHI_0, wl=pro3D3_C2_atm.axes['wavelength'],
                                    NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                    sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                                    XBLOCK = xblock, XGRID = xgrid, progress=False,
                                    OUTPUT_LAYERS=3, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_1to4_F_atm = S3DF.run(THVDEG=THETA_0, wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP,
                             atm=pro3D3_C2_atm, sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                             XBLOCK = xBLOCK, XGRID = xGRID, OUTPUT_LAYERS=3, DEPO=0)
norm_C2_1to4_F_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_1to4_F_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_1to4_F_atm
norm = norm_C2_1to4_F_atm
for i in range (0, 4):
    TCASE = int(i+1)
    indZA = round(TCASE - 1)
    print("TESTCASE:", TCASE)
    print_C2_res_atm(m, norm, TCASE, grid3_sensors, U_sign=-1, v_sign=1,
                     mI=m['I_down (0+)'][:,indZA], mQ=m['Q_down (0+)'][:,indZA],
                     mU=m['U_down (0+)'][:,indZA], mV=m['V_down (0+)'][:,indZA])

#### Case 5 - 9

##### Run

In [ ]:
xgrid_sensors = np.linspace(0., 7., 71)*scale
ygrid_sensors = np.linspace(0., 7., 71)*scale
zgrid_sensors = np.array([0., 1., 2., 3., 4., 5.])*scale
grid3_sensors = Grid3D(xgrid_sensors, ygrid_sensors, zgrid_sensors, periodic=True)

# Placement of sensors
POSZ = grid3_sensors.zGRID[-1]-1e-6

print("POSZ = ", POSZ)

# Sun position
THETA_0    = 40.
PHI_0    = 180.

THETA_0_bis = 180.- THETA_0
PHI_0_bis = 180.- PHI_0

# !!!! grid3 is different than sensors grid !!!
x0, y0, sensors, icells = create_sensors(grid3_sensors, POSZ=POSZ, THDEG=THETA_0_bis, PHDEG=PHI_0_bis, FOV=0., LOC='ATMOS',
                                         CELL_SIZE=grid3_sensors.xgrid[1]-grid3_sensors.xgrid[0], grid3D_atm=grid3)

THETA = np.array([180., 140., 140., 140., 140.])
PHI = np.array([0., 0., 60., 120., 180.])

le     = {'th_deg':np.array(180.-THETA),
        'phi_deg':np.array(PHI+180.),
        'count_level':np.array([0, 0, 0, 0, 0]),
        'zip':True}

if FIND_OPTIMAL_XB_XG:
    k_time = np.inf
    for xgrid in XG:
        for xblock in XB:
                m_test = S3DF.run(THVDEG=THETA_0, wl=pro3D3_C2_atm.axes['wavelength'],
                                    NBPHOTONS=CHECK_NBPHOTONS, NBLOOP=CHECK_NBLOOP, atm=pro3D3_C2_atm,
                                    sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                                    XBLOCK = xblock, XGRID = xgrid, progress=False,
                                    OUTPUT_LAYERS=1, DEPO=0)
                if float(m_test.attrs['kernel time (s)']) < k_time:
                    k_time = float(m_test.attrs['kernel time (s)'])
                    best_xb = xblock
                    best_xg = xgrid
                print("time (s) =", m_test.attrs['kernel time (s)'], "; xblock =", xblock, "; xgrid =", xgrid)
    print("\n\nBest xblock =", best_xb, " : best xgrid =", best_xg)
    xBLOCK = best_xb
    xGRID = best_xg
else:
    xBLOCK = XBLOCK
    xGRID = XGRID

m3D_C2_5to9_F_atm = S3DF.run(THVDEG=THETA_0, wl=pro3D3_C2_atm.axes['wavelength'], NBPHOTONS=NBPHOTONS, NBLOOP=NBLOOP,
                atm=pro3D3_C2_atm, sensor=sensors, le=le, surf=surf_C2, NF=NTH,
                XBLOCK = xBLOCK, XGRID = xGRID, OUTPUT_LAYERS=1, DEPO=0)
norm_C2_5to9_F_atm = np.cos(np.radians(THETA_0))/np.pi
print("kernel time (s) =", "{:.2f}".format(float(m3D_C2_5to9_F_atm.attrs['kernel time (s)'])))

##### Results

In [ ]:
m = m3D_C2_5to9_F_atm
norm = norm_C2_5to9_F_atm
for i in range (0, 5):
    TCASE = int(i+5)
    indZA = round(TCASE - 5)
    print("TESTCASE:", TCASE)
    print_C2_res_atm(m, norm, TCASE, grid3_sensors, U_sign=-1, v_sign=1,
                     mI=m['I_up (TOA)'][:,indZA], mQ=m['Q_up (TOA)'][:,indZA],
                     mU=m['U_up (TOA)'][:,indZA], mV=m['V_up (TOA)'][:,indZA])